## Installation

In [1]:
!pip -q install transformers datasets evaluate transformers[torch]
! pip install -U accelerate
!pip install bitsandbytes
!pip install rouge_score
!pip install huggingface-hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer

from peft import LoraConfig, get_peft_model, TaskType
from peft import PeftModel, PeftConfig

import warnings
warnings.filterwarnings('ignore')

2024-12-06 13:14:53.991991: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-06 13:14:54.025463: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-06 13:14:55.574769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load Dataset

In [18]:
from datasets import load_dataset

billsum = load_dataset("billsum", split='train')

In [19]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 18949
})

In [20]:
billsum = billsum.train_test_split(test_size=0.2)

In [21]:
billsum["train"][0]

{'text': "SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Long-Term Care Trust Account Act of \n2006''.\n\nSEC. 2. LONG-TERM CARE TRUST ACCOUNTS.\n\n    (a) In General.--Subchapter F of chapter 1 of the Internal Revenue \nCode of 1986 (relating to exempt organizations) is amended by adding at \nthe end the following new part:\n\n                ``PART IX--LONG-TERM CARE TRUST ACCOUNTS\n\n``SEC. 530A. LONG-TERM CARE TRUST ACCOUNTS.\n\n    ``(a) General Rule.--A Long-Term Care Trust Account shall be exempt \nfrom taxation under this subtitle. Notwithstanding the preceding \nsentence, such account shall be subject to the taxes imposed by section \n511 (relating to imposition of tax on unrelated business income of \ncharitable organizations).\n    ``(b) Long-Term Care Trust Account.--For purposes of this section, \nthe term `Long-Term Care Trust Account' means a trust created or \norganized in the United States for the exclusive benefit of an \nindividual who is the designate

## Tokenization and Pre-processing

In [57]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [45]:
print(len(tokenizer.encode(text)))  # Check input token length


6599


In [58]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [59]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/3790 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-large-cnn")

In [9]:
import evaluate

rouge = evaluate.load("rouge")

In [60]:
!huggingface-cli login --token xyz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `podcast` has been saved to /home/khairnar.as/.cache/huggingface/stored_tokens
Your token has been saved to /home/khairnar.as/.cache/huggingface/token
Login successful.
The current active token is: `podcast`


In [26]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Train model

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=64,                       
    lora_alpha=64,               
    lora_dropout=0.01,          
    bias="all",                 
    task_type=TaskType.SEQ_2_SEQ_LM,
    target_modules=['q_proj', 'v_proj']
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

training_args = Seq2SeqTrainingArguments(
    output_dir="bart-large-cnn-repo",
    eval_strategy="epoch",
    auto_find_batch_size=True,
    learning_rate=2e-4,
    per_device_train_batch_size=16,  # Adjusted batch size
    per_device_eval_batch_size=16,   # Adjusted batch size
    gradient_accumulation_steps = 1,
    # optim = "paged_adamw_32bit",
    save_steps = 0,
    weight_decay=0.01,
    logging_steps = 25,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
    bf16=True,
    push_to_hub=True,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = "cosine"
)


trainer = Seq2SeqTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=2844, training_loss=1.522106369168782, metrics={'train_runtime': 4988.7446, 'train_samples_per_second': 9.116, 'train_steps_per_second': 0.57, 'total_flos': 1.0119026606447002e+17, 'train_loss': 1.522106369168782, 'epoch': 3.0})

In [28]:
lora_model.print_trainable_parameters()

trainable params: 9,771,008 || all params: 415,727,616 || trainable%: 2.3503
